# Devin's Notebook

In [48]:
from collections import defaultdict, Counter
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
from random import choice, sample

### 1. Load Training Data

In [2]:
with open("train.txt") as file :
    all_lines = file.read().splitlines()

followers = Counter()
following = Counter()
edges = defaultdict(list)
all_nodes = []
source_nodes = []

for line in tqdm(all_lines) :
    nodes = line.split("\t")
    source, sinks = nodes[0], nodes[1:]
    sinks = list(map(int, sinks))
    source = int(source)
    while source in sinks: sinks.remove(source)
        
    edges[source] = sinks
    following[source] = len(sinks)
    for sink in sinks :
        followers[sink] += 1
    
    all_nodes.append(source)
    all_nodes.extend(sinks)
    source_nodes.append(source)
    
all_nodes = list(set(all_nodes))

100%|██████████| 20000/20000 [00:31<00:00, 640.24it/s] 


Stats about `train.txt`

In [3]:
i = 0
null = 0

for node in following :
    if following[node] == 0 :
        null += 1
    else :
        i += following[node]

print("Total lines in train.txt: ", len(all_lines))
print("Total edges in train.txt: ", i)
print("Total distinct nodes", len(all_nodes))
print("Size of Source nodes: ", len(set(source_nodes)))
print("Size of null entries: ", null)

Total lines in train.txt:  20000
Total edges in train.txt:  24004344
Total distinct nodes 4867136
Size of Source nodes:  20000
Size of null entries:  430


In [ ]:
testData_nodes = list(testData['Source'])
testData_nodes.extend(list(testData['Sink']))
testData_nodes = set(testData_nodes)
print(len(testData_nodes))

### 2. Data Analysis

Checking if all nodes in the test set (source and sink) appear in the training set.

In [57]:
# i = 0; j = 0;
# for node in testData['Source'] :
#     if node in source_nodes :
#         i += 1
#     elif node in all_nodes :
#         j += 1

# print("Source nodes on the test set was in source(train):", i, "; sinks(train):", j)

# i = 0; j = 0;
# for node in testData['Sink'] :
#     if node in source_nodes :
#         i += 1
#     elif node in all_nodes :
#         j += 1
        
# print("Sink nodes on the test set was in source(train):", i, "; sinks(train):", j)

Source nodes on the test set was in source(train): 2000 ; sinks(train): 0
Sink nodes on the test set was in source(train): 376 ; sinks(train): 1624


Checking if sink node exist in source node's sinks

In [81]:
# j = []
# for i, source in enumerate(testData['Source']) :
#     j.append(len(edges[source]))

# print(collections.Counter(j).most_common(10))

[(1, 18), (52, 15), (34, 14), (39, 12), (23, 12), (6, 12), (24, 12), (68, 12), (26, 11), (2, 11)]


### 3. Preparing Data for Training

#### 3.1 Positive Data

In [5]:
node_list_1 = []
node_list_2 = []

for source in edges :
    for sink in edges[source] :
        node_list_1.append(source)
        node_list_2.append(sink)

#### 3.2 Negative Data

In [30]:
neg_edges = defaultdict(list)

for source in edges :
    while len(neg_edges[source]) < 200:
        sink = choice(all_nodes)
        if (sink not in edges[source] and sink not in neg_edges[source] and sink != source) :
            neg_edges[source].append(sink)
              
neg_node_list_1 = []
neg_node_list_2 = []

for source in neg_edges :
    for sink in neg_edges[source] :
        neg_node_list_1.append(source)
        neg_node_list_2.append(sink)

***
**WARNING: TAKES 4 HOURS**
***

In [74]:
# k = []
# j = []

# for node in tqdm(testData_nodes, total=len(testData_nodes)) :
#     tmp_k = [i for i, value in enumerate(node_list_1) if value == node]
#     k.extend(tmp_k)
    
#     tmp_j = [i for i, value in enumerate(node_list_2) if value == node]
#     j.extend(tmp_j)
    
# print(len(k))
# print(k[0:9])
# print(len(j))
# print(j[0:9])

100%|██████████| 3948/3948 [3:26:20<00:00,  3.14s/it]  

4697704
[19062765, 19062766, 19062767, 19062768, 19062769, 19062770, 19062771, 19062772, 19062773]
434262
[748342, 1063577, 1268103, 1872727, 1920705, 2136447, 2159034, 3139726, 3379748]


In [1]:
# newkj = k.copy()
# newj = j.copy()
# newkj.extend(newj)
# indexes = list(set(newkj))
# indexes.sort()
# print(len(indexes), min(indexes), max(indexes), sum(indexes)/len(indexes))

NameError: name 'k' is not defined

In [108]:
# """
# x1 = []
# x2 = []
# x3 = []
# x4 = []

# for node in tqdm(node_list_1, total=len(node_list_1)):
#     x1.append(nfollowing(row['Source'], edges))
#     x3.append(nfollowers(row['Source'], edges))
    
# for node in tqdm(node_list_2, total=len(node_list_2)):
#     x2.append(nfollowing(row['Sink'], edges))
#     x4.append(nfollowers(row['Sink'], edges))
# """
# trainData = pd.DataFrame({'Source': node_list_1, 'Sink': node_list_2})
# trainData_reduced = trainData.iloc[indexes,:]
# trainData_reduced = trainData_reduced.drop_duplicates()
# print(trainData_reduced)
# trainData_reduced.to_csv('trainData_reduced.txt', sep="\t", index=True, header=True, index_label = "Id")

           Source     Sink
0          540762  1912140
1          540762  1537559
2          540762  3091331
3          540762  2757277
4          540762  3237295
...           ...      ...
24004339  3547024  1075576
24004340  3547024  4549841
24004341  3547024  1135647
24004342  3547024   807274
24004343  3547024  3897045

[5053079 rows x 2 columns]


In [53]:
# trainData_reduced = pd.read_csv('trainData_reduced.txt', sep="\t")
# print(trainData_reduced.head())
# print(trainData_reduced.shape)

   Id  Source     Sink
0   0  540762  1912140
1   1  540762  1537559
2   2  540762  3091331
3   3  540762  2757277
4   4  540762  3237295
(5053079, 3)


### 4. Feature Engineering

- **x1** - Number following (Source).
- **x2** - Number following (Sink).
- **x3** - Number of followers (Source).
- **x4** - Number of followers (Sink).
- **x5** - Common nodes - following (Source and sink).

In [101]:
## functions

In [24]:
def generate_features (source_list, sink_list, following, followers, target=None) :
    x1 = []
    x2 = []
    x3 = []
    x4 = []
    x5 = []
    
    for i in tqdm(range(len(source_list))) :
        source = source_list[i]
        sink = sink_list[i]
        x1.append(following[source])
        x2.append(following[sink])
        x3.append(followers[source])
        x4.append(followers[sink])
    
    if target is not None :
        df = pd.DataFrame({'Source': source_list, 'Sink': sink_list, 'x1': x1, 'x2': x2, 'x3': x3, 'x4': x4, 'Target': target})
    else :
        df = pd.DataFrame({'Source': source_list, 'Sink': sink_list, 'x1': x1, 'x2': x2, 'x3': x3, 'x4': x4})
        
    return df

In [31]:
pos_trainData = generate_features(node_list_1, node_list_2, following, followers, 1)
print(pos_trainData.head())
print(pos_trainData.shape)

100%|██████████| 24004344/24004344 [00:46<00:00, 520948.50it/s]


   Source     Sink   x1   x2   x3   x4  Target
0  540762  1912140  143   20  142   37       1
1  540762  1537559  143   79  142   76       1
2  540762  3091331  143   26  142   28       1
3  540762  2757277  143  222  142  107       1
4  540762  3237295  143  119  142   57       1
(24004344, 7)


In [32]:
neg_trainData = generate_features(neg_node_list_1, neg_node_list_2, following, followers, 0)
print(neg_trainData.head())
print(neg_trainData.shape)

100%|██████████| 4000000/4000000 [00:08<00:00, 493742.38it/s]


   Source     Sink   x1  x2   x3  x4  Target
0  540762  3798563  143   0  142   1       0
1  540762  2024044  143   0  142   2       0
2  540762  2256179  143   0  142   6       0
3  540762  3133206  143  14  142   3       0
4  540762   877004  143   0  142   2       0
(4000000, 7)


In [40]:
trainData_full = pd.concat([pos_trainData, neg_trainData], ignore_index=True)
trainData_features = trainData_full[['x1', 'x2', 'x3', 'x4']]
trainData_response = trainData_full[['Target']]
print(trainData_features.head())
print(trainData_features.shape)

print(trainData_response.head())
print(trainData_response.shape)

    x1   x2   x3   x4
0  143   20  142   37
1  143   79  142   76
2  143   26  142   28
3  143  222  142  107
4  143  119  142   57
(28004344, 4)
   Target
0       1
1       1
2       1
3       1
4       1
(28004344, 1)


### 5. Training Model

In [46]:
def mean_squared_error(y_true, y_pred):
    np.asarray(y_true)
    np.asarray(y_pred)
    return np.mean((y_pred - y_true)**2)

In [42]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(trainData_features, trainData_response, test_size=0.2, random_state=90051)
print("Training set has {} instances. Test set has {} instances.".format(X_train.shape[0], X_test.shape[0]))

Training set has 22403475 instances. Test set has 5600869 instances.


In [53]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression().fit(X_train, y_train)
y_pred_train = lin_reg.predict(X_train)
y_pred_test = lin_reg.predict(X_test)

print('Train MSE:', mean_squared_error(y_pred_train, y_train))
print('Test MSE:', mean_squared_error(y_pred_test, y_test))

Train MSE: Target    0.109942
dtype: float64
Test MSE: Target    0.110046
dtype: float64


### 6. Loading Test Data

In [54]:
testData = pd.read_csv('test-public.txt', sep="\t")
testData_ids = list(testData['Id'])
print(testData.head())
print(testData.shape)

   Id   Source     Sink
0   1  3563811  3600160
1   2  2052043  1401960
2   3  4517994  1690636
3   4  1660006  4349447
4   5   581111  1882617
(2000, 3)


In [55]:
testData_features = generate_features(list(testData['Source']), list(testData['Sink']), following, followers, None)
print(testData_features.head())
print(testData_features.shape)

100%|██████████| 2000/2000 [00:00<00:00, 20673.56it/s]

    Source     Sink   x1  x2  x3  x4
0  3563811  3600160   21   0   3  29
1  2052043  1401960   71   0  13   9
2  4517994  1690636  205   0  80  17
3  1660006  4349447  506   0  32  36
4   581111  1882617   18   0   5  46
(2000, 6)


### 7. New Prediction

In [68]:
X_new = testData_features[['x1', 'x2', 'x3', 'x4']]
y_new = lin_reg.predict(X_new)

In [69]:
output = pd.DataFrame({'Id': testData_ids, 'Predicted': y_new.flatten()})
output.head

<bound method NDFrame.head of         Id  Predicted
0        1   0.766499
1        2   0.762016
2        3   0.772171
3        4   0.771837
4        5   0.771544
...    ...        ...
1995  1996   0.760392
1996  1997   0.775787
1997  1998   0.759215
1998  1999   0.759612
1999  2000   0.759658

[2000 rows x 2 columns]>

In [71]:
output.to_csv('output.csv', sep=",", index=False, header=True)